## Step 01: Import pyspark and initialize Spark 

In [2]:
!pip install pyspark
# create entry points to spark
from pyspark import SparkContext, SparkConf # Spark
from pyspark.sql import SparkSession # Spark SQL

spark_conf = SparkConf()
# setup application name
spark_conf.setAppName("Text Analyzer")
# setup working processors
spark_conf.setMaster('local[*]')

# We add this line to avoid an error : "Cannot run multiple SparkContexts at once". 
# If there is an existing spark context, we will reuse it instead of creating a new context.
sc = SparkContext.getOrCreate()

# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext(conf = spark_conf)
spark = SparkSession(sparkContext=sc)

## Step 02: Create RDD and display total number of lines

In [3]:
# load both book into rdd
rdd_sh = sc.textFile("Scrum Handbook.txt")
rdd_ap = sc.textFile("Agile Processes  in Software Engineering  and Extreme Programming.txt")

# count the number of lines in each rdd
line_count_scrum = rdd_sh.count()
line_count_agile = rdd_ap.count()

# display 
print('Total number of lines in <Scrum Handbook>: ',  line_count_scrum)
print('Total number of lines in <Agile Processess in Software Engineering and Extreme Programming>: ', line_count_agile)

Total number of lines in <Scrum Handbook>:  4617
Total number of lines in <Agile Processess in Software Engineering and Extreme Programming>:  21569


In [4]:
rdd_sh.collect()

["Jeff Sutherland's ",
 '',
 'Scrum Handbook ',
 '',
 '',
 '',
 '',
 'Everything ',
 'you need ',
 'to know ',
 'to start ',
 '',
 'a Scrum project ',
 'in your ',
 'organization ',
 '',
 'SCRUM ',
 '',
 'TRAINING ',
 '',
 'INSTITUTE ',
 '',
 'm w press ',
 '',
 '',
 '',
 'This book is dedicated to Nobel Laureate Muhammad Yunus and the ',
 'Grameen Bank for originating microenterprise development and the ',
 "Accion International President's Advisory Board, responsible for much of ",
 'microenterprise development in the western hemisphere. ',
 '',
 'The strategy for bootstrapping the poor out of poverty has been ',
 'a model for freeing hundreds of thousands of software developers from ',
 'developer abuse caused by poor management practices. ',
 '',
 'Thanks to the reviewers of the text who include among many others: ',
 '',
 '• Tom Poppendieck ',
 '',
 '• Hen rick Kniberg ',
 '',
 '• Rowan Bunning ',
 '',
 '• Clifford Thompson ',
 '',
 '',
 '',
 'About this book ',
 '',
 'This manual

## Step 03: Cleaning/Manipulating text
### Assumption : Since this task aims for text analyzing, i assume empty line shouldn't be considered "words", thus they were cleaned at this stage

In [5]:
# import re module
import re

In [6]:
# define a function to clean the data
def text_cleaning(x):
    # get rid of non-alphabetic letter in data
    output_with_only_aplha_n_space = re.sub('[^a-zA-Z ]+', '', x)    
    # change all uppercase letters to lowercase
    output_with_lowercase = output_with_only_aplha_n_space.lower()
    # get rid of leading and trailing spaces
    output_without_useless_spaces = output_with_lowercase.strip()  
    # return cleaned data
    return output_without_useless_spaces

In [7]:
# call text_cleaning function in map to clean the data and filter off the empty lines
rdd_sh_cleaned = rdd_sh.map(text_cleaning).filter(lambda line: len(line) > 0)
rdd_ap_cleaned = rdd_ap.map(text_cleaning).filter(lambda line: len(line) > 0)
# display the text data after cleaning
rdd_sh_cleaned.collect()
#print(rdd_ap_cleaned.take(10))

['jeff sutherlands',
 'scrum handbook',
 'everything',
 'you need',
 'to know',
 'to start',
 'a scrum project',
 'in your',
 'organization',
 'scrum',
 'training',
 'institute',
 'm w press',
 'this book is dedicated to nobel laureate muhammad yunus and the',
 'grameen bank for originating microenterprise development and the',
 'accion international presidents advisory board responsible for much of',
 'microenterprise development in the western hemisphere',
 'the strategy for bootstrapping the poor out of poverty has been',
 'a model for freeing hundreds of thousands of software developers from',
 'developer abuse caused by poor management practices',
 'thanks to the reviewers of the text who include among many others',
 'tom poppendieck',
 'hen rick kniberg',
 'rowan bunning',
 'clifford thompson',
 'about this book',
 'this manual is based on the scrum papers published by the scrum',
 'training institute see wwwscrumtraininginstitutecom',
 'for information on how to receive your own

## Step04: Transforming Data and Counting the words
### Also as previous step, empty space are not considered to be word, which are cleaned at this step.

In [8]:
# split data by space, transform into key value pair and reduce by key, also get rid of empty word in this step
rdd_sh_splitted = rdd_sh_cleaned.flatMap(lambda x : x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda a,b: a + b).filter(lambda word: len(word[0]) != 0)
# display each word and number count
print('Frequency of each word: ', rdd_sh_splitted.collect())
print('-----------------------------------------------------')
# display the top 20 most frequent word
rdd_sh_sorted = rdd_sh_splitted.sortBy(lambda x:x[1], ascending = False)
print('The 20 most frequent words: ', rdd_sh_sorted.take(20))
print('-----------------------------------------------------')
# split data by space, transform into key value pair and reduce by key
rdd_ap_splitted = rdd_ap_cleaned.flatMap(lambda x : x.split(" ")).map(lambda x : (x,1)).reduceByKey(lambda a,b: a + b).filter(lambda word: len(word[0]) != 0)
# display each word and number count
print('Frequency of each word: ', rdd_ap_splitted.collect())
print('-----------------------------------------------------')
# display the top 20 most frequent word
rdd_ap_sorted = rdd_ap_splitted.sortBy(lambda x:x[1], ascending = False)
print('The 20 most frequent words: ', rdd_ap_sorted.take(20))

Frequency of each word:  [('jeff', 48), ('sutherlands', 34), ('scrum', 395), ('handbook', 34), ('everything', 6), ('you', 17), ('need', 9), ('to', 477), ('know', 10), ('start', 6), ('a', 451), ('project', 92), ('in', 360), ('your', 6), ('organization', 19), ('training', 10), ('institute', 11), ('m', 4), ('w', 2), ('press', 5), ('this', 142), ('book', 13), ('is', 348), ('dedicated', 7), ('nobel', 1), ('laureate', 1), ('muhammad', 1), ('yunus', 1), ('and', 534), ('the', 1238), ('grameen', 1), ('bank', 1), ('for', 195), ('originating', 1), ('microenterprise', 2), ('development', 98), ('accion', 1), ('international', 3), ('presidents', 2), ('advisory', 1), ('board', 3), ('responsible', 9), ('much', 26), ('of', 537), ('western', 1), ('hemisphere', 1), ('strategy', 3), ('bootstrapping', 1), ('poor', 5), ('out', 19), ('poverty', 1), ('has', 53), ('been', 27), ('model', 17), ('freeing', 1), ('hundreds', 2), ('thousands', 3), ('software', 52), ('developers', 23), ('from', 80), ('developer', 5),

The 20 most frequent words:  [('the', 1238), ('of', 537), ('and', 534), ('to', 477), ('a', 451), ('scrum', 395), ('in', 360), ('is', 348), ('team', 260), ('product', 232), ('for', 195), ('that', 181), ('it', 165), ('on', 149), ('sprint', 146), ('this', 142), ('with', 132), ('as', 124), ('are', 119), ('by', 118)]
-----------------------------------------------------
Frequency of each word:  [('lnbip', 58), ('i', 418), ('helen', 10), ('sharp', 53), ('tracy', 7), ('hall', 45), ('eds', 80), ('agile', 748), ('processes', 58), ('in', 3076), ('software', 922), ('engineering', 154), ('and', 3965), ('extreme', 29), ('programming', 79), ('th', 49), ('international', 97), ('conference', 97), ('xp', 146), ('edinburgh', 3), ('uk', 59), ('may', 103), ('proceedings', 72), ('springer', 50), ('open', 77), ('lecture', 2), ('notes', 23), ('business', 141), ('information', 118), ('processing', 8), ('series', 21), ('editors', 9), ('wil', 1), ('van', 18), ('der', 2), ('aalst', 1), ('eindhoven', 2), ('techni

The 20 most frequent words:  [('the', 8145), ('and', 3965), ('of', 3944), ('to', 3714), ('in', 3076), ('a', 2701), ('is', 1539), ('that', 1355), ('for', 1194), ('as', 1028), ('on', 1015), ('we', 975), ('with', 969), ('software', 922), ('this', 915), ('are', 784), ('agile', 748), ('it', 741), ('development', 737), ('was', 711)]


## Step05: Remove stop words

In [9]:
# install nltk module
!pip3 install nltk
# import nltk module 
import nltk
from nltk.corpus import stopwords
# download stopwords
nltk.download('stopwords')

     |████████████████████████████████| 1.5MB 1.0MB/s eta 0:00:01     |███████████▎                    | 512kB 1.1MB/s eta 0:00:01     |███████████████████▏            | 870kB 1.1MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449908 sha256=88e0a83575e17dfc185acfbd13362011934d97086444b0f2b1d6cc73ee9bd1fe
  Stored in directory: /home/student/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/student/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# set up filter words as stopwords from nltk
filter_words = stopwords.words('english')
# filter the unsorted RDD from last step
rdd_sh_sw_removed = rdd_sh_sorted.filter(lambda x: x[0] not in filter_words).filter(lambda word: word!='\t')
# print(rdd_sh_sw_removed.take(10))
# count unique words
unique_sh = rdd_sh_sw_removed.count()
print("There are ", unique_sh, " unique words in <Scrum Handbook>")

# filter the unsorted RDD from last step
rdd_ap_sw_removed = rdd_ap_sorted.filter(lambda x: x[0] not in filter_words)
# print(rdd_ap_sw_removed.take(10))
# count unique words
unique_ap = rdd_ap_sw_removed.count()
print("There are ", unique_ap, "unique words in <Agile Processes in Software Engineering and Extreme Programming>")

There are  2936  unique words in <Scrum Handbook>
There are  9611 unique words in <Agile Processes in Software Engineering and Extreme Programming>


In [11]:
print(rdd_ap_cleaned.flatMap(lambda x: x.split(" ")).count())
print(rdd_sh_sw_removed.take(10))

135211
[('scrum', 395), ('team', 260), ('product', 232), ('sprint', 146), ('development', 98), ('teams', 94), ('project', 92), ('owner', 85), ('work', 79), ('backlog', 74)]


## Step 06: Find average occurrence of a word

In [12]:
# use the cleaned data that without stopwords to get the total number of words 
total_sh = rdd_sh_sw_removed.collect()
no_total_sh = 0
for word in total_sh:
    no_total_sh = no_total_sh + word[1]
print("The number of total words in <Scrum Handbook> is: ", no_total_sh)
print("Thus the average occurrences of the words is: ",no_total_sh,"/",unique_sh," which equals", no_total_sh/unique_sh)

print('--------------------------------------')
# use the cleaned data that without stopwords to get the total number of words 
total_ap = rdd_ap_sw_removed.collect()
no_total_ap = 0
for word in total_ap:
    no_total_ap = no_total_ap + word[1]
print("The number of total words in <Agile Processess in Software Engineering and Extreme Programming> is: ", no_total_ap)
print("Thus the average occurrences of the words is: ",no_total_ap,"/",unique_ap," which equals", no_total_ap/unique_ap)

The number of total words in <Scrum Handbook> is:  10810
Thus the average occurrences of the words is:  10810 / 2936  which equals 3.681880108991826
--------------------------------------
The number of total words in <Agile Processess in Software Engineering and Extreme Programming> is:  79056
Thus the average occurrences of the words is:  79056 / 9611  which equals 8.225574862137135


## Step07: Visualization

In [13]:
# install matplotlib module
!pip3 install matplotlib

     |████████████████████████████████| 13.1MB 1.1MB/s eta 0:00:01    |█▏                              | 481kB 941kB/s eta 0:00:14     |███                             | 1.2MB 941kB/s eta 0:00:13     |████▊                           | 1.9MB 1.0MB/s eta 0:00:11     |█████▊                          | 2.3MB 1.0MB/s eta 0:00:11     |███████▋                        | 3.1MB 3.0MB/s eta 0:00:04     |██████████▏                     | 4.2MB 1.0MB/s eta 0:00:09     |██████████▌                     | 4.3MB 1.0MB/s eta 0:00:09     |████████████████▎               | 6.7MB 1.0MB/s eta 0:00:07     |███████████████████▊            | 8.1MB 1.1MB/s eta 0:00:05     |█████████████████████▎          | 8.7MB 522kB/s eta 0:00:09     |████████████████████████        | 9.9MB 1.2MB/s eta 0:00:03     |████████████████████████▋       | 10.1MB 1.2MB/s eta 0:00:03     |██████████████████████████▋     | 10.9MB 1.2MB/s eta 0:00:02     |███████████████████████████▍    | 11.2MB 1.2MB/s eta 0:00:02     |████████████████

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt
frequency_sh = rdd_sh_sw_removed.collect()
# print(frequency_sh)
rank = 0
x_axis_sh = []
y_axis_sh = []
# iterate through the list to get the rank and number of occurrences as x and y 
for word in frequency_sh:
    rank = rank + 1
    occurrence = word[1]
    x_axis_sh.append(rank)
    y_axis_sh.append(occurrence)
    
frequency_ap = rdd_ap_sw_removed.collect()
# print(frequency_sh)
rank = 0
x_axis_ap = []
y_axis_ap = []
# iterate through the list to get the rank and number of occurrences as x and y 
for word in frequency_ap:
    rank = rank + 1
    occurrence = word[1]
    x_axis_ap.append(rank)
    y_axis_ap.append(occurrence)
    
    
fig = plt.figure(figsize=(10,6))
plt.plot(x_axis_sh, y_axis_sh, label="Scrum Handbook")
plt.plot(x_axis_ap, y_axis_ap, label="Agile Processes in Software Engineering and Extreme Programming")
plt.xscale('log')
plt.yscale('log')
plt.yticks([1,10,100,1000,10000])
plt.legend(loc = 'upper right')
plt.title("Word Distribution")
plt.xlabel("rank")
plt.ylabel("Number of occurrences")
plt.show()

## Explanation
### From what we can observe from the plot:
### After both data set have been cleaned and stop words been removed, there are around 3 thousand word in Scrum Handbook, which have a number of occurrences range from 1 to less than 1000, while there are about 10 thousand unique word in Agile Processes in Software Engineering and Extreme Programming, which results in a distribution range from 1 to about 1300.
### We can conclude from the comparison that there are cleary more unique word in Agile Processes than in Scrum Handbook, which also reasonably result in a higher occurrence for words with the same rank

In [ ]:
x_axis_sh = []
y_axis_sh = []
x_axis_ap = []
y_axis_ap = []
for i in range(15):
    x_axis_sh.append(frequency_sh[i][0])
    y_axis_sh.append(frequency_sh[i][1])
    x_axis_ap.append(frequency_ap[i][0])
    y_axis_ap.append(frequency_ap[i][1])
fig = plt.figure(figsize=(10,20))
ax_scrum = fig.add_subplot(2,1,1)
ax_agile = fig.add_subplot(2,1,2)
ax_scrum.plot(x_axis_sh, y_axis_sh, label="Scrum Handbook", c = 'red')
ax_agile.plot(x_axis_ap, y_axis_ap, label="Agile Processes in Software Engineering and Extreme Programming", c = 'yellow')
ax_scrum.set_title("Top 15 most frequent words in Scrum Handbook")
ax_agile.set_title("Top 15 most frequent words in Agile Processes")
ax_scrum.tick_params(labelrotation=90)
ax_scrum.set_xlabel('Word')
ax_scrum.set_ylabel('Number of occurrence')
ax_agile.tick_params(labelrotation=90)
ax_agile.set_xlabel('Word')
ax_agile.set_ylabel('Number of occurrence')